In [ ]:
# yolo_v1_full.py

import os
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.ops import nms
from torch.utils.data import DataLoader
from torchvision.datasets import VOCDetection


VOC_CLASSES = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person',
    'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]
class_to_idx = {cls_name: i for i, cls_name in enumerate(VOC_CLASSES)}

# ------------------------------------------------------------------------------
# 1. Configuration
# ------------------------------------------------------------------------------
class Config:
    S = 7               # Grid size
    B = 2               # Bounding boxes per cell
    C = 20              # Classes (Pascal VOC has 20)
    IMAGE_SIZE = 448    # Input 이미지 크기
    IMAGE_CH_SIZE = 3
    BATCH_SIZE = 16
    LR = 1e-4
    EPOCHS = 50 #3 # 50
    CONF_THRESHOLD = 0.2
    NMS_IOU_THRESH = 0.4
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


# 논문에 제시된 아키텍처 구성 (YOLOv1)
architecture_config = [
    (7, 64, 2, 3),       # (kernel_size, filters, stride, padding)
    "M",                 # maxpool
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],  # 해당 블록을 4번 반복
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],  # 해당 블록을 2번 반복
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1)
]

# ------------------------------------------------------------------------------
# 2. Model Definition (YOLOv1)
# ------------------------------------------------------------------------------
def create_conv_layers(config, in_channels):
    layers = []
    for module in config:
        if type(module) == tuple:
            # 튜플 형태: (kernel_size, filters, stride, padding)
            kernel_size, filters, stride, padding = module
            layers.append(nn.Conv2d(in_channels, filters, kernel_size, stride, padding))
            layers.append(nn.LeakyReLU(0.1))
            in_channels = filters
        elif module == "M":
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        elif type(module) == list:
            # 리스트 형태: [ conv1 튜플, conv2 튜플, 반복 횟수 ]
            conv1, conv2, num_repeats = module
            for _ in range(num_repeats):
                # 첫 번째 컨볼루션
                k, f, s, p = conv1
                layers.append(nn.Conv2d(in_channels, f, k, s, p))
                layers.append(nn.LeakyReLU(0.1))
                in_channels = f
                # 두 번째 컨볼루션
                k, f, s, p = conv2
                layers.append(nn.Conv2d(in_channels, f, k, s, p))
                layers.append(nn.LeakyReLU(0.1))
                in_channels = f
    return nn.Sequential(*layers)

class YOLOv1(nn.Module):
    def __init__(self, in_channels=3, S=7, B=2, C=20): # split_size=7, num_boxes=2, num_classes=20
        super(YOLOv1, self).__init__()
        self.features = create_conv_layers(architecture_config, in_channels)
        # 입력 이미지가 448x448인 경우, 마지막 컨볼루션 feature map은 7x7 (논문 기준)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * 7 * 7, 4096),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),  # 논문에서 사용한 dropout
            nn.Linear(4096, S * S * (C + B * 5))
        )
        self.S = S
        self.B = B
        self.C = C

    def forward(self, x):
        # x: [B,3,H,W] → features → [B,1024,S,S]
        x = self.features(x)
        # classifier → [B, S*S*(5B + C)]
        x = self.classifier(x)
        # reshape → [B, S, S, 5B + C]
        # print('YOLOv1 forward', x.view(-1, self.S, self.S, 5*self.B + self.C).size())
        # YOLOv1 forward torch.Size([16, 7, 7, 30])
        return x.view(-1, self.S, self.S, 5*self.B + self.C) # 계산 복잡도를 낮추기 위해 (N, S, S, 5*B+C) 형태로 반환한다.


# ------------------------------------------------------------------------------
# 3. Loss Function (YOLOv1 original)
# ------------------------------------------------------------------------------
# 주어진 두 박스의 중심 좌표와 크기를 바탕으로 좌측상단, 우측하단 좌표를 계산하고, 교집합 영역을 통해 IoU를 계산합니다.
# IoU 계산 함수 (YOLOv1에서 사용하는 bounding box 형식: [x_center, y_center, width, height])
def iou(boxes1, boxes2, eps=1e-6):
    """
    boxes1, boxes2: 텐서, 마지막 차원이 [x_center, y_center, width, height]
    """
    # 좌측 상단, 우측 하단 좌표 계산. (x1,y1,x2,y2)로 변환
    box1_x1 = boxes1[:,0] - boxes1[:,2] / 2
    box1_y1 = boxes1[:,1] - boxes1[:,3] / 2
    box1_x2 = boxes1[:,0] + boxes1[:,2] / 2
    box1_y2 = boxes1[:,1] + boxes1[:,3] / 2

    box2_x1 = boxes2[:,0] - boxes2[:,2] / 2
    box2_y1 = boxes2[:,1] - boxes2[:,3] / 2
    box2_x2 = boxes2[:,0] + boxes2[:,2] / 2
    box2_y2 = boxes2[:,1] + boxes2[:,3] / 2

    # 교집합 영역
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)
    inter_w  = (x2 - x1).clamp(min=0)
    inter_h  = (y2 - y1).clamp(min=0)
    inter = inter_w * inter_h

    # 합집합 영역
    box1_area = torch.abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = torch.abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))
    union = box1_area + box2_area - inter + eps

    # IoU: Intersaction over Union
    iou_val = inter / union
    return iou_val

class YoloLoss(nn.Module):
    def __init__(self, S, B, C, lambda_coord=5, lambda_noobj=0.5):
        super().__init__()
        self.S, self.B, self.C = S, B, C
        self.mse = nn.MSELoss(reduction='sum')
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, pred, target):
        """
        pred, target: [N, S, S, 5B+C]
        target 포맷: 각 셀 [x, y, w, h, conf, one-hot-class(C)]
        """
        # print('YoloLoss forward', pred.shape) # (16, 7, 7, 30)
        B, S, _, _ = pred.shape
        coord_loss = 0
        obj_loss = 0
        noobj_loss = 0
        class_loss = 0

        # 순회 대신 벡터화 가능하지만, 가독성을 위해 loop 사용
        for i in range(S):
            for j in range(S):
                # target confidence = 1인 셀들
                obj_mask = target[:,i,j,4] == 1
                noobj_mask = target[:,i,j,4] == 0

                # 해당 셀의 예측 박스와 타깃 분리
                pred_cell = pred[:, i, j, :5*self.B].view(-1, self.B, 5) # (N, B, 5). 즉, (16, 2, 5)
                # print('pred_cell: ', pred_cell.shape)
                true_cell = target[:, i, j, :5] # (N, 5)

                # ====================
                # 0) 책임 박스 좌표
                # ====================
                if obj_mask.any():
                    # 책임 박스 선정: 각 샘플마다 IoU가 최대인 박스 인덱스
                    # pred_cell[obj_mask]: [N_obj, B, 5]
                    pred_boxes = pred_cell[obj_mask, :, :4] # (N_obj, B, 4)
                    # (N_obj, 4) -> (N_obj, 1, 4) -> (N_obj, B, 4): prediction은 박스가 2개이상 일 수 있으니, 계산 용이성을 위해 target도 shape을 똑같이 맞추어 준다.
                    true_boxes = true_cell[obj_mask, :4].unsqueeze(1).expand_as(pred_boxes)
                    # IoU 계산 후 argmax. 두박스 모두 (N * B, 4)이며 리턴은 (N_obj, B) 이다
                    ious = iou(pred_boxes.reshape(-1, 4), true_boxes.reshape(-1, 4)).view(-1, self.B)
                    best_idx = torch.argmax(ious, dim=1) # (N_obj,): B개의 박스 중 최고를 선택한다.

                    # 책임 박스 좌표
                    n_obj = best_idx.size(0) # N_obj
                    batch_idx = torch.arange(n_obj, device=pred.device) # [0, ..., N_obj - 1]
                    pred_chosen = pred_boxes[batch_idx, best_idx] # (N_obj, 4)
                    true_chosen = true_boxes[batch_idx, best_idx] # (N_obj, 4)
                    # print('pred_boxes:', pred_boxes.shape)
                    # print('ious:', ious)
                    # print('best_idx:', best_idx)
                    # print('n_obj:', n_obj)
                    # print('batch_idx:', batch_idx)

                # ====================
                # 1) 좌표 손실 (object 셀)
                # ====================
                if obj_mask.any():
                    # 예측에서 B개 박스 중 책임 존재하는 박스 사용
                    # 크기 루트 비교
                    pred_xy = pred_chosen[:, :2]
                    true_xy = true_chosen[:, :2]
                    coord_loss += self.mse(pred_xy, true_xy)
                    pred_wh = pred_chosen[:, 2:4].clamp(min=1e-6)
                    true_wh = true_chosen[:, 2:4]  # target은 음수가 없으므로 clamp 불필요
                    coord_loss += self.mse(torch.sqrt(pred_wh), torch.sqrt(true_wh))
                    # print('coord_loss: ', coord_loss)

                # ====================
                # 2) Obejct Confidence 손실
                # ====================
                # object 셀
                if obj_mask.any():
                    # Object confidence 손실
                    pred_conf = pred_cell[obj_mask, :, 4]      # (N_obj, B)
                    # 선택된 conf만 1, 나머지는 noobj 손실에 포함
                    chosen_conf = pred_conf[batch_idx, best_idx] # (N_obj,)
                    # print('chosen_conf: ', chosen_conf.shape, chosen_conf)
                    obj_loss += self.mse(chosen_conf, torch.ones_like(chosen_conf))

                    # 나머지 박스는 no-object 손실: 오브젝트가 없다면 해당 loss가 0에 수렴해야 좋은 것임
                    noobj_conf_mask = torch.ones_like(pred_conf, dtype=torch.bool) # (N_obj, B)
                    noobj_conf_mask[batch_idx, best_idx] = False
                    noobj_conf = pred_conf[noobj_conf_mask]
                    noobj_loss += self.mse(noobj_conf, torch.zeros_like(noobj_conf))
                # no-object 셀 전체 박스
                if noobj_mask.any():
                    noobj_pred = pred_cell[noobj_mask, :, 4]  # (N_noobj, B)
                    noobj_loss += self.mse(noobj_pred,  torch.zeros_like(noobj_pred))
                # print('noobj_loss: ', noobj_loss)

                # ====================
                # 3) Class 손실
                # ====================
                if obj_mask.any():
                    pred_cls = pred[obj_mask, i, j, 5:]
                    true_cls = target[obj_mask, i, j, 5:]
                    class_loss += self.mse(pred_cls, true_cls)
                    # print('class_loss: ', class_loss)

        total_loss = (
            self.lambda_coord * coord_loss +
            obj_loss +
            self.lambda_noobj * noobj_loss +
            class_loss
        )
        # print('total_loss: ', total_loss)
        return total_loss / B


# ------------------------------------------------------------------------------
# 4. Dataset & Dataloader 예시 (Pascal VOC)
# ------------------------------------------------------------------------------
class VOCDataset(torch.utils.data.Dataset):
    def __init__(self, root, year='2007', image_set='train', S=7, B=2, C=20, transform=None):
        self.dataset = VOCDetection(root, year=year, image_set=image_set, download=True)
        self.S, self.B, self.C = S, B, C
        self.transform = transform
        self.class_to_idx = class_to_idx

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, target = self.dataset[idx]
        boxes = []
        labels = []
        for obj in target['annotation']['object']:
            bbox = obj['bndbox']
            # 원본 좌표 [1..W/H] → normalized [0..1]
            x1 = float(bbox['xmin']) / img.width
            y1 = float(bbox['ymin']) / img.height
            x2 = float(bbox['xmax']) / img.width
            y2 = float(bbox['ymax']) / img.height
            boxes.append([x1,y1,x2,y2])
            cls_name = obj['name']
            labels.append(self.class_to_idx[cls_name])

        if self.transform:
            img = self.transform(img)

        # target tensor: [S, S, 5B + C], 초기 0
        target_tensor = torch.zeros((self.S, self.S, 5*self.B + self.C))
        cell_size = 1.0 / self.S

        for box, cls in zip(boxes, labels):
            x1,y1,x2,y2 = box
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1

            i = int(y_center / cell_size)
            j = int(x_center / cell_size)
            # cell 내 상대 좌표
            dx = (x_center - j*cell_size) / cell_size
            dy = (y_center - i*cell_size) / cell_size

            # 첫 번째 박스 책임 할당
            target_tensor[i,j,0:4] = torch.tensor([dx, dy, w, h])
            target_tensor[i,j,4] = 1
            target_tensor[i,j,5+cls] = 1

        return img, target_tensor


# ------------------------------------------------------------------------------
# 5. Post-processing: Decode + NMS
# ------------------------------------------------------------------------------

# 1) Decode 단계: 모델 출력 → 바운딩박스, 점수, 클래스 리스트로 변환
def decode_predictions(output, conf_thresh, S, B, C):
    """
    output: [N, S, S, 5B + C]
    returns:
      batch_boxes   : list of N tensors [M_i, 4]  (x1, y1, x2, y2)
      batch_scores  : list of N tensors [M_i]     (score)
      batch_classes : list of N tensors [M_i]     (class_idx)
    """
    N = output.size(0)
    cell_size = 1.0 / S

    batch_boxes, batch_scores, batch_classes = [], [], []
    for b in range(N):
        preds = output[b]         # [S, S, 5B+C]
        boxes, scores, classes = [], [], []

        for i in range(S):
            for j in range(S):
                cell = preds[i, j]             # [5B + C]
                class_probs = cell[5*B:]       # [C]
                for bi in range(B):
                    bx, by, bw, bh, conf = cell[bi*5 : bi*5+5]
                    class_scores = conf * class_probs
                    max_conf, cls = torch.max(class_scores, dim=0)

                    if max_conf > conf_thresh:
                        # 격자→절대 좌표 변환
                        x_c = (j + bx) * cell_size
                        y_c = (i + by) * cell_size
                        x1 = x_c - bw/2;  y1 = y_c - bh/2
                        x2 = x_c + bw/2;  y2 = y_c + bh/2

                        boxes.append([x1, y1, x2, y2])
                        scores.append(max_conf)
                        classes.append(cls)

        if boxes:
            batch_boxes.append(torch.tensor(boxes))
            batch_scores.append(torch.stack(scores))
            batch_classes.append(torch.tensor(classes, dtype=torch.long))
        else:
            batch_boxes.append(torch.zeros((0,4)))
            batch_scores.append(torch.zeros((0,)))
            batch_classes.append(torch.zeros((0,), dtype=torch.long))

    return batch_boxes, batch_scores, batch_classes

# 2) NMS 단계: 클래스별로 Non-Maximum Suppression 적용
def apply_nms(batch_boxes, batch_scores, batch_classes, iou_thresh):
    """
    batch_boxes, batch_scores, batch_classes: decode_predictions 반환값
    returns: list of N tensors [K_i, 6] → (x1,y1,x2,y2,score,cls)
    """
    batch_dets = []

    for boxes, scores, classes in zip(batch_boxes, batch_scores, batch_classes):
        if boxes.numel() == 0:
            batch_dets.append(torch.zeros((0,6)))
            continue

        kept = []
        for cls_id in classes.unique():
            mask = (classes == cls_id)
            cls_boxes  = boxes[mask]
            cls_scores = scores[mask]
            keep_idxs  = nms(cls_boxes, cls_scores, iou_thresh)

            if keep_idxs.numel() > 0:
                selected = torch.cat([
                    cls_boxes[keep_idxs],
                    cls_scores[keep_idxs].unsqueeze(1),
                    torch.full((keep_idxs.numel(),1), cls_id, dtype=torch.float)
                ], dim=1)  # [k,6]
                kept.append(selected)

        if kept:
            batch_dets.append(torch.vstack(kept))
        else:
            batch_dets.append(torch.zeros((0,6)))

    return batch_dets

# 3) Wrapper
def postprocess(output, conf_thresh, iou_thresh, S, B, C):
    boxes, scores, classes = decode_predictions(output, conf_thresh, S, B, C)
    return apply_nms(boxes, scores, classes, iou_thresh)


# ------------------------------------------------------------------------------
# 6. Training & Validation Loop
# ------------------------------------------------------------------------------
def train_one_epoch(model, loader, loss_fn, opt, device):
    model.train()
    total_loss = 0
    for imgs, targets in tqdm(loader, desc='Train batchs'):
        imgs, targets = imgs.to(device), targets.to(device)
        preds = model(imgs)
        # print('train_one_epoch', preds.shape)
        loss = loss_fn(preds, targets)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def validate(model, loader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for imgs, targets in loader:
            imgs, targets = imgs.to(device), targets.to(device)
            preds = model(imgs)
            loss = loss_fn(preds, targets)
            total_loss += loss.item()
    return total_loss / len(loader)


# ------------------------------------------------------------------------------
# 7. Main: 학습 및 추론 예시
# ------------------------------------------------------------------------------
# def main():

cfg = Config()
# transforms
transform = T.Compose([
    T.Resize((cfg.IMAGE_SIZE, cfg.IMAGE_SIZE)),
    T.ToTensor()
])
# dataset & loader
train_ds = VOCDataset(root='./data', image_set='train', transform=transform, S=cfg.S, B=cfg.B, C=cfg.C)
val_ds   = VOCDataset(root='./data', image_set='val',   transform=transform, S=cfg.S, B=cfg.B, C=cfg.C)
train_loader = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=cfg.BATCH_SIZE)

# 모델·손실·최적화기
model = YOLOv1(cfg.IMAGE_CH_SIZE, cfg.S, cfg.B, cfg.C).to(cfg.DEVICE)
loss_fn = YoloLoss(cfg.S, cfg.B, cfg.C).to(cfg.DEVICE)
opt = optim.Adam(model.parameters(), lr=cfg.LR)

# 학습 루프
for epoch in range(1, cfg.EPOCHS+1):
    train_loss = train_one_epoch(model, train_loader, loss_fn, opt, cfg.DEVICE)
    val_loss   = validate(model, val_loader, loss_fn, cfg.DEVICE)
    print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# 추론 예시
test_img, _ = val_ds[0]
test_pred = model(test_img.unsqueeze(0).to(cfg.DEVICE))
detections = postprocess(test_pred, cfg.CONF_THRESHOLD, cfg.NMS_IOU_THRESH, cfg.S, cfg.B, cfg.C)
print("Sample detections:", detections[0])

# if __name__ == "__main__":
#     main()

'''
Train batchs: 100%|██████████| 157/157 [01:48<00:00,  1.44it/s]
Epoch 01 | Train Loss: 9.5173 | Val Loss: 7.8427
Train batchs: 100%|██████████| 157/157 [01:49<00:00,  1.44it/s]
Epoch 02 | Train Loss: 8.2690 | Val Loss: 7.7727
Train batchs: 100%|██████████| 157/157 [01:51<00:00,  1.41it/s]
Epoch 03 | Train Loss: 298.4498 | Val Loss: 8.7577
Train batchs: 100%|██████████| 157/157 [01:51<00:00,  1.41it/s]
Epoch 04 | Train Loss: 8.8979 | Val Loss: 7.8043
Train batchs: 100%|██████████| 157/157 [01:51<00:00,  1.41it/s]
Epoch 05 | Train Loss: 8.4618 | Val Loss: 7.7567
Train batchs: 100%|██████████| 157/157 [01:49<00:00,  1.43it/s]
Epoch 06 | Train Loss: 8.3562 | Val Loss: 7.7235
'''
print()



In [43]:
# 추론 예시
#val_ds   = VOCDataset(root='./data', image_set='val',   transform=transform, S=cfg.S, B=cfg.B, C=cfg.C)
#val_loader   = DataLoader(val_ds,   batch_size=cfg.BATCH_SIZE)
test_img, _ = val_ds[0]
test_pred = model(test_img.unsqueeze(0).to(cfg.DEVICE))
detections = postprocess(test_pred, cfg.CONF_THRESHOLD, cfg.NMS_IOU_THRESH, cfg.S, cfg.B, cfg.C)
print("Sample detections:", detections[0])


Sample detections: tensor([], size=(0, 6))


## 삭제된 코드 Snippet (사용하지 마세요)

In [ ]:
# ------------------------------------------------------------------------------
# 2. Model Definition (YOLOv1)
# ------------------------------------------------------------------------------
class YOLOv1_OLD(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super().__init__()
        self.S, self.B, self.C = S, B, C

        # 1) Feature Extractor (Darknet-style)
        def conv(in_ch, out_ch, k, s=1, p=0):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, k, s, p),
                nn.LeakyReLU(0.1, inplace=True)
            )

        self.features = nn.Sequential(
            conv(3,   64, 7, 2, 3), nn.MaxPool2d(2,2),
            conv(64,  192, 3, 1, 1), nn.MaxPool2d(2,2),

            conv(192, 128, 1),       conv(128, 256, 3,1,1),
            conv(256, 256, 1),       conv(256, 512, 3,1,1),
            nn.MaxPool2d(2,2),

            *self._make_layers(512, 256, 512, repeats=4),
            nn.MaxPool2d(2,2),

            *self._make_layers(512, 512, 1024, repeats=2),
            nn.MaxPool2d(2,2),

            conv(1024, 1024, 3,1,1),
            conv(1024, 1024, 3,2,1),
            conv(1024, 1024, 3,1,1),
            conv(1024, 1024, 3,1,1),
        )

        # 2) Detection Head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * S * S, 4096),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, S * S * (5*B + C))
        )

    def _make_layers(self, in_ch, mid_ch, out_ch, repeats):
        layers = []
        for _ in range(repeats):
            layers += [
                nn.Sequential(
                    nn.Conv2d(in_ch, mid_ch, 1),
                    nn.LeakyReLU(0.1, inplace=True)
                ),
                nn.Sequential(
                    nn.Conv2d(mid_ch, out_ch, 3,1,1),
                    nn.LeakyReLU(0.1, inplace=True)
                )
            ]
            in_ch = out_ch
        return layers

    def forward(self, x):
        # x: [B,3,H,W] → features → [B,1024,S,S]
        x = self.features(x)
        # classifier → [B, S*S*(5B + C)]
        x = self.classifier(x)
        # reshape → [B, S, S, 5B + C]
        return x.view(-1, self.S, self.S, 5*self.B + self.C)


# ------------------------------------------------------------------------------
# 5. Post-processing: Decode + NMS
# ------------------------------------------------------------------------------
def decode_and_nms(output, conf_thresh, iou_thresh, S, B, C):
    """
    output: [N, S, S, 5B + C]
    return: list of detections per image (x1,y1,x2,y2,score,cls)
    """
    batch = []
    cell_size = 1.0 / S
    for b in range(output.size(0)):
        preds = output[b]  # [S,S,5B+C]
        boxes_all = []
        for i in range(S):
            for j in range(S):
                cell = preds[i,j]
                class_probs = cell[5*B:]
                for bi in range(B):
                    bx,by,bw,bh,conf = cell[bi*5:bi*5+5]
                    score = conf * class_probs
                    max_conf, cls = torch.max(score, dim=-1)
                    if max_conf > conf_thresh:
                        x_center = (j + bx) * cell_size
                        y_center = (i + by) * cell_size
                        w, h = bw, bh
                        x1 = x_center - w/2
                        y1 = y_center - h/2
                        x2 = x_center + w/2
                        y2 = y_center + h/2
                        boxes_all.append([x1,y1,x2,y2, max_conf.item(), cls.item()])

        if not boxes_all:
            batch.append(torch.zeros((0,6)))
            continue

        boxes = torch.tensor(boxes_all)
        keep = []
        # 클래스별 NMS
        for c in range(C):
            mask = boxes[:,5] == c
            if mask.sum() == 0:
                continue
            cls_boxes = boxes[mask]
            coords = cls_boxes[:,:4]
            scores = cls_boxes[:,4]
            idxs = nms(coords, scores, iou_thresh)
            keep.append(cls_boxes[idxs])
        if keep:
            batch.append(torch.vstack(keep))
        else:
            batch.append(torch.zeros((0,6)))
    return batch